# Text Classification: Natural Language Processing with Disaster Tweets

## Downloading the Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
## read csv data using pandas
train_df = pd.read_csv('train.csv',            # make sure to put the file in the working directory
                 sep=',',                      # separater
                 header = 0,                   # the header that indicates the column names is at row 0
                 usecols=['id','keyword', 'location', 'text', 'target'])

# print out the dataframe
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


## Basic Text Preprocessing

We'll first create text_target_train_data and text_target_test_data so that we have versions of the training and testing datasets that only have the text, and not the id, keyword, or location.

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [4]:
drop_cols = ['id', 'keyword', 'location']

text_target_train_data = train_df.drop(drop_cols, axis = 1)

In [5]:
text_target_train_data

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


Now, we use the nltk (tokenizer) for removing stopwords, stemming, and deleting other common Twitter-specific elements (such as user tags and links).

In [6]:
import nltk   # tokenizer
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer

import string
import re   # regular expressions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'\w+')

# Given Twitter data, we also want to delete user tags and links
tags = r'@\w*'
html = r'<.*?>'
urls = r'https?://\S+|www\.\S+'
non_ascii_pattern = r'[^\x00-\x7F]'

In [8]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text

In [9]:
def preprocess_text(sentence, stem=False):
    s = re.sub(tags, "", sentence)
    s = re.sub(html, "", s)
    s = re.sub(urls, "", s)
    s = [re.sub(non_ascii_pattern, "", s)]

    sentence_cleaned = []

    sentence = s[0]
    punctuation_removed = remove_punctuation(sentence)

    words = punctuation_removed.lower().split()

    for word in words:
        if word not in stopwords and not any(char.isdigit() for char in word):
            if stem:
                sentence_cleaned.append(stemmer.stem(word).lower())
            else:
                sentence_cleaned.append(word.lower())

    return tokenizer.tokenize(" ".join(sentence_cleaned))

In [10]:
print(f"Orignal: {text_target_train_data.text[7609]}")
print(f"Orignal: {text_target_train_data.target[7609]}")    # a disaster tweet
print()
print(f"Preprocessed: {preprocess_text(text_target_train_data.text[7609])}")

Orignal: @aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.
Orignal: 1

Preprocessed: ['control', 'wild', 'fires', 'california', 'even', 'northern', 'part', 'state', 'troubling']


In [11]:
print(f"Orignal: {text_target_train_data.text[569]}")
print(f"Orignal: {text_target_train_data.target[569]}")    # not a disaster tweet
print()
print(f"Preprocessed: {preprocess_text(text_target_train_data.text[569])}")

Orignal: STAR WARS POWER OF THE JEDI COLLECTION 1 BATTLE DROID HASBRO - Full read by eBay http://t.co/yI30ZgiZsW http://t.co/2jGVhw7YZs
Orignal: 0

Preprocessed: ['star', 'wars', 'power', 'jedi', 'collection', 'battle', 'droid', 'hasbro', 'full', 'read', 'ebay']


In [12]:
text_target_train_data.text = text_target_train_data.text.map(preprocess_text)

In [13]:
text_target_train_data.head()

,text,target
0,"[deeds, reason, earthquake, may, allah, forgiv...",1
1,"[forest, fire, near, la, ronge, sask, canada]",1
2,"[residents, asked, shelter, place, notified, o...",1
3,"[people, receive, wildfires, evacuation, order...",1
4,"[got, sent, photo, ruby, alaska, smoke, wildfi...",1


## Training Conventional Models

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [15]:
# Extract text data from the 'text' column
text_data_train = text_target_train_data['text'].apply(lambda x: ' '.join(x))

# Convert text data to a list of strings
text_only_train_data = text_data_train.tolist()

# Fit CountVectorizer on the training data
vectorizer = CountVectorizer(max_features=2150)    # size of vocabulary fine-tuned
vectorizer.fit(text_only_train_data)

CountVectorizer(max_features=2150)

In [17]:
# Get the vocabulary learned by CountVectorizer
vocabulary = vectorizer.get_feature_names_out()

Since the testing set doesn't have target labels (which may make it difficult to evaluate the model's performance, let us split the training set into a training and validation set.

In [18]:
# Transform text data using the vectorizer
X_train_bow = vectorizer.transform(text_only_train_data)
y_train = text_target_train_data['target']

X_main, X_test, y_main, y_test = train_test_split(X_train_bow, y_train, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.2, random_state=42)

print(X_train.shape)
print(len(y_train))

(4872, 2150)
4872


## Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [21]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_train_pred = logreg.predict(X_train)
y_val_pred = logreg.predict(X_val)
y_test_pred = logreg.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      2765
           1       0.92      0.83      0.87      2107

    accuracy                           0.90      4872
   macro avg       0.90      0.89      0.89      4872
weighted avg       0.90      0.90      0.90      4872

              precision    recall  f1-score   support

           0       0.78      0.85      0.81       703
           1       0.76      0.67      0.72       515

    accuracy                           0.77      1218
   macro avg       0.77      0.76      0.76      1218
weighted avg       0.77      0.77      0.77      1218

              precision    recall  f1-score   support

           0       0.80      0.85      0.83       874
           1       0.78      0.71      0.75       649

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.79      1523
weighted avg       0.79      0.79      0.79      1523



### With L2 Regularization

We can see that the precision, recall, and f1-scores are alright, but not very high. Note that the size of the Bag of Words representation vector increases with the size of the vocabulary! In other words, all words will be 14824 dimensional, which means that sparsity may be a problem. Sparse data can lead to the model overfitting, because the model learns patterns specific to the training data and fails to generalize to our unseen (validation) data.

We can try to tune our selection of hyperparameter alpha to see if our model improves. We take 6 alpha values and examine which value for alpha results in the best model performance.

In [22]:
regularization_strengths = [0.01, 0.1, 0.25, 0.5, 1, 5]

best_score = 0
best_C = None

for C in regularization_strengths:
    logreg_with_l2 = LogisticRegression(max_iter=1000, penalty='l2', C=C)

    logreg_with_l2.fit(X_train, y_train)

    y_val_pred = logreg_with_l2.predict(X_val)

    score = logreg_with_l2.score(X_val, y_val)
    print(f"C={C}, Validation Accuracy: {score}")

    if score > best_score:
        best_score = score
        best_C = C

print(f"Best C: {best_C}, Best Validation Accuracy: {best_score}")

C=0.01, Validation Accuracy: 0.7118226600985221
C=0.1, Validation Accuracy: 0.7963875205254516
C=0.25, Validation Accuracy: 0.7963875205254516
C=0.5, Validation Accuracy: 0.7873563218390804
C=1, Validation Accuracy: 0.7742200328407225
C=5, Validation Accuracy: 0.764367816091954
Best C: 0.1, Best Validation Accuracy: 0.7963875205254516


In [24]:
logreg_with_l2 = LogisticRegression(max_iter=1000, penalty='l2', solver='saga', C=best_C)
logreg_with_l2.fit(X_train, y_train)

y_train_pred = logreg_with_l2.predict(X_train)
y_val_pred = logreg_with_l2.predict(X_val)
y_test_pred = logreg_with_l2.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.81      0.95      0.87      2765
           1       0.92      0.70      0.79      2107

    accuracy                           0.84      4872
   macro avg       0.86      0.82      0.83      4872
weighted avg       0.85      0.84      0.84      4872

              precision    recall  f1-score   support

           0       0.78      0.91      0.84       703
           1       0.84      0.64      0.73       515

    accuracy                           0.80      1218
   macro avg       0.81      0.78      0.78      1218
weighted avg       0.80      0.80      0.79      1218

              precision    recall  f1-score   support

           0       0.77      0.92      0.84       874
           1       0.85      0.64      0.73       649

    accuracy                           0.80      1523
   macro avg       0.81      0.78      0.78      1523
weighted avg       0.81      0.80      0.79      1523



### With SVD (for Dimensionality Reduction)

In [25]:
# Convert text data to a list of strings
train_data_for_svd = text_data_train.tolist()

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

In [27]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(train_data_for_svd)

In [28]:
svd = TruncatedSVD(n_components=500)
normalizer = Normalizer(copy=False)

X_svd = svd.fit_transform(tfidf_matrix)
X_svd = normalizer.fit_transform(X_svd)
y_svd = text_target_train_data['target']

In [29]:
print(X_svd.shape)
print(len(y_svd))

(7613, 500)
7613


In [30]:
X_main_svd, X_test_svd, y_main_svd, y_test_svd = train_test_split(X_svd, y_svd, test_size=0.2, random_state=42)
X_train_svd, X_val_svd, y_train_svd, y_val_svd = train_test_split(X_main_svd, y_main_svd, test_size=0.2, random_state=42)

In [31]:
logreg.fit(X_train_svd, y_train_svd)

y_train_pred2 = logreg.predict(X_train_svd)
y_val_pred2 = logreg.predict(X_val_svd)
y_test_pred2 = logreg.predict(X_test_svd)

# Evaluate the performance
print(classification_report(y_train_svd, y_train_pred2))
print(classification_report(y_val_svd, y_val_pred2))
print(classification_report(y_test_svd, y_test_pred2))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86      2765
           1       0.86      0.74      0.79      2107

    accuracy                           0.84      4872
   macro avg       0.84      0.82      0.83      4872
weighted avg       0.84      0.84      0.83      4872

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       703
           1       0.79      0.70      0.74       515

    accuracy                           0.79      1218
   macro avg       0.79      0.78      0.79      1218
weighted avg       0.79      0.79      0.79      1218

              precision    recall  f1-score   support

           0       0.79      0.86      0.83       874
           1       0.79      0.70      0.74       649

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



### With PCA (for Dimensionality Reduction)

In [32]:
train_data_for_pca = text_data_train.tolist()

In [33]:
from sklearn.decomposition import PCA

In [34]:
# Initialize PCA with desired number of components
pca = PCA(n_components=500)

# Fit the model with data and apply the dimensionality reduction
X_pca = pca.fit_transform(tfidf_matrix.toarray())
y_pca = text_target_train_data['target']

In [35]:
print(X_pca.shape)
print(len(y_pca))

(7613, 500)
7613


In [36]:
X_main_pca, X_test_pca, y_main_pca, y_test_pca = train_test_split(X_pca, y_pca, test_size=0.2, random_state=42)
X_train_pca, X_val_pca, y_train_pca, y_val_pca = train_test_split(X_main_pca, y_main_pca, test_size=0.2, random_state=42)

In [37]:
logreg.fit(X_train_pca, y_train_pca)

y_train_pred3 = logreg.predict(X_train_pca)
y_val_pred3 = logreg.predict(X_val_pca)
y_test_pred3 = logreg.predict(X_test_pca)

# Evaluate the performance
print(classification_report(y_train_pca, y_train_pred3))
print(classification_report(y_val_pca, y_val_pred3))
print(classification_report(y_test_pca, y_test_pred3))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86      2765
           1       0.90      0.66      0.76      2107

    accuracy                           0.82      4872
   macro avg       0.84      0.80      0.81      4872
weighted avg       0.83      0.82      0.82      4872

              precision    recall  f1-score   support

           0       0.78      0.91      0.84       703
           1       0.84      0.64      0.73       515

    accuracy                           0.80      1218
   macro avg       0.81      0.78      0.78      1218
weighted avg       0.80      0.80      0.79      1218

              precision    recall  f1-score   support

           0       0.77      0.91      0.84       874
           1       0.84      0.64      0.73       649

    accuracy                           0.80      1523
   macro avg       0.81      0.77      0.78      1523
weighted avg       0.80      0.80      0.79      1523



## Naive Bayes Classifier

In [38]:
from sklearn.naive_bayes import MultinomialNB

In [39]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

y_train_pred = naive_bayes.predict(X_train)
y_val_pred = naive_bayes.predict(X_val)
y_test_pred = naive_bayes.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.84      0.91      0.87      2765
           1       0.87      0.77      0.81      2107

    accuracy                           0.85      4872
   macro avg       0.85      0.84      0.84      4872
weighted avg       0.85      0.85      0.85      4872

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       703
           1       0.78      0.71      0.74       515

    accuracy                           0.79      1218
   macro avg       0.79      0.78      0.79      1218
weighted avg       0.79      0.79      0.79      1218

              precision    recall  f1-score   support

           0       0.79      0.86      0.83       874
           1       0.79      0.70      0.74       649

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



## SVM

In [40]:
from sklearn.svm import SVC

We can first try an SVM model with a linear kernel, since it is computationally efficient and often a good starting point. However, from our previous analysis with the logistic regression and Naive Bayes classifiers, we may infer that the data is not linearly separable, and thus the performance may not be very good.

In [41]:
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)

y_train_pred = svm_linear.predict(X_train)
y_val_pred = svm_linear.predict(X_val)
y_test_pred = svm_linear.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      2765
           1       0.93      0.87      0.90      2107

    accuracy                           0.92      4872
   macro avg       0.92      0.91      0.91      4872
weighted avg       0.92      0.92      0.92      4872

              precision    recall  f1-score   support

           0       0.78      0.82      0.80       703
           1       0.73      0.68      0.71       515

    accuracy                           0.76      1218
   macro avg       0.76      0.75      0.75      1218
weighted avg       0.76      0.76      0.76      1218

              precision    recall  f1-score   support

           0       0.79      0.80      0.79       874
           1       0.73      0.71      0.72       649

    accuracy                           0.76      1523
   macro avg       0.76      0.75      0.76      1523
weighted avg       0.76      0.76      0.76      1523



Instead, now let us try to use a Radial Basis Function (RBF) kernel. We typically use this kernel when the decision boundary is highly non-linear or not well-defined. Note that the gamma parameter controls the smoothness of the decision boundary. Smaller values of gamma lead to smoother decision boundaries.

In [42]:
svm_rbf = SVC(kernel='rbf', gamma='scale')
svm_rbf.fit(X_train, y_train)

y_train_pred = svm_rbf.predict(X_train)
y_val_pred = svm_rbf.predict(X_val)
y_test_pred = svm_rbf.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      2765
           1       0.97      0.86      0.91      2107

    accuracy                           0.93      4872
   macro avg       0.94      0.92      0.93      4872
weighted avg       0.93      0.93      0.93      4872

              precision    recall  f1-score   support

           0       0.78      0.90      0.83       703
           1       0.83      0.65      0.73       515

    accuracy                           0.79      1218
   macro avg       0.80      0.77      0.78      1218
weighted avg       0.80      0.79      0.79      1218

              precision    recall  f1-score   support

           0       0.78      0.89      0.83       874
           1       0.82      0.66      0.73       649

    accuracy                           0.79      1523
   macro avg       0.80      0.78      0.78      1523
weighted avg       0.80      0.79      0.79      1523



## Decision Trees Classifier

In [43]:
from sklearn import tree

In [44]:
dec_trees = tree.DecisionTreeClassifier()
dec_trees.fit(X_train, y_train)

y_train_pred = dec_trees.predict(X_train)
y_val_pred = dec_trees.predict(X_val)
y_test_pred = dec_trees.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      2765
           1       1.00      0.97      0.98      2107

    accuracy                           0.98      4872
   macro avg       0.99      0.98      0.98      4872
weighted avg       0.98      0.98      0.98      4872

              precision    recall  f1-score   support

           0       0.78      0.78      0.78       703
           1       0.70      0.70      0.70       515

    accuracy                           0.74      1218
   macro avg       0.74      0.74      0.74      1218
weighted avg       0.74      0.74      0.74      1218

              precision    recall  f1-score   support

           0       0.73      0.71      0.72       874
           1       0.62      0.64      0.63       649

    accuracy                           0.68      1523
   macro avg       0.67      0.68      0.68      1523
weighted avg       0.68      0.68      0.68      1523



## Random Forest Classifier

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

y_train_pred = random_forest.predict(X_train)
y_val_pred = random_forest.predict(X_val)
y_test_pred = random_forest.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      2765
           1       0.99      0.97      0.98      2107

    accuracy                           0.98      4872
   macro avg       0.98      0.98      0.98      4872
weighted avg       0.98      0.98      0.98      4872

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       703
           1       0.73      0.71      0.72       515

    accuracy                           0.77      1218
   macro avg       0.76      0.76      0.76      1218
weighted avg       0.77      0.77      0.77      1218

              precision    recall  f1-score   support

           0       0.77      0.78      0.77       874
           1       0.70      0.68      0.69       649

    accuracy                           0.74      1523
   macro avg       0.73      0.73      0.73      1523
weighted avg       0.74      0.74      0.74      1523



## Gradient Boosting Classifier

In [47]:
from sklearn.ensemble import GradientBoostingClassifier

In [48]:
grad_boost = GradientBoostingClassifier()
grad_boost.fit(X_train, y_train)

y_train_pred = grad_boost.predict(X_train)
y_val_pred = grad_boost.predict(X_val)
y_test_pred = grad_boost.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.72      0.97      0.83      2765
           1       0.93      0.51      0.66      2107

    accuracy                           0.77      4872
   macro avg       0.83      0.74      0.74      4872
weighted avg       0.81      0.77      0.76      4872

              precision    recall  f1-score   support

           0       0.70      0.95      0.81       703
           1       0.87      0.46      0.60       515

    accuracy                           0.74      1218
   macro avg       0.79      0.70      0.70      1218
weighted avg       0.77      0.74      0.72      1218

              precision    recall  f1-score   support

           0       0.71      0.93      0.80       874
           1       0.84      0.47      0.61       649

    accuracy                           0.74      1523
   macro avg       0.77      0.70      0.71      1523
weighted avg       0.76      0.74      0.72      1523



In [49]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [100, 200, 300],    # Number of boosting stages
    'learning_rate': [0.1, 0.15, 0.2],  # Learning rate
    'max_depth': [5, 8, 10]             # Max depth of individual trees
}

gb_classifier = GradientBoostingClassifier()

# Perform grid search using cross-validation
grid_search = GridSearchCV(estimator=gb_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters found:", grid_search.best_params_)

# Evaluate the model with the best parameters on the test set
best_model = grid_search.best_estimator_
val_accuracy = best_model.score(X_val, y_val)
print("Val accuracy with best parameters:", val_accuracy)

Best parameters found: {'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 300}
Val accuracy with best parameters: 0.7758620689655172


In [54]:
# Tuned these parameters via gridsearch
grad_boost_tuned = GradientBoostingClassifier(
    n_estimators=grid_search.best_params_['n_estimators'],
    learning_rate=grid_search.best_params_['learning_rate'],
    max_depth=grid_search.best_params_['max_depth'])
grad_boost_tuned.fit(X_train, y_train)

y_train_pred = grad_boost_tuned.predict(X_train)
y_val_pred = grad_boost_tuned.predict(X_val)
y_test_pred = grad_boost_tuned.predict(X_test)

# Evaluate the performance
print(classification_report(y_train, y_train_pred))
print(classification_report(y_val, y_val_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      2765
           1       0.98      0.85      0.91      2107

    accuracy                           0.93      4872
   macro avg       0.94      0.92      0.93      4872
weighted avg       0.94      0.93      0.93      4872

              precision    recall  f1-score   support

           0       0.77      0.88      0.82       703
           1       0.79      0.64      0.71       515

    accuracy                           0.78      1218
   macro avg       0.78      0.76      0.77      1218
weighted avg       0.78      0.78      0.77      1218

              precision    recall  f1-score   support

           0       0.77      0.85      0.81       874
           1       0.77      0.66      0.71       649

    accuracy                           0.77      1523
   macro avg       0.77      0.76      0.76      1523
weighted avg       0.77      0.77      0.77      1523

